In [14]:
#Setup pandas and spreadsheet
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format
spreadsheet = "FPL.xlsx"

#Create players dataframe of all players
player_dfs = pd.read_excel(spreadsheet, sheet_name=[i for i in range(21)], index_col=0)
players_verbose = pd.concat([player_dfs[i] for i in range(21)])
players = players_verbose[['Quality', 'Pos.', 'Team', 'Price']]

#Create teams quality dataframe
teams_df = pd.read_excel(spreadsheet, sheet_name="Teams", index_col=0)
teams = teams_df[["Attacking Quality", "Defensive Quality"]]

#Get upcoming gameweek
games_df = pd.read_excel(spreadsheet, sheet_name="Games")
gameweek = games_df["GW"].max() + 1

#Get fixtures
fixtures = pd.read_excel(spreadsheet, sheet_name="Fixtures")


In [15]:
def get_upcoming_fixtures(team, look_ahead, current=gameweek):
    team_fixtures = fixtures[["GW", team]]
    gws = team_fixtures["GW"]
    teams = team_fixtures[team]

    upcoming_fixtures = team_fixtures.loc[gws <= current + look_ahead]
    upcoming_fixtures = upcoming_fixtures.loc[gws >= current]
    upcoming_fixtures = upcoming_fixtures.loc[teams.notnull()]

    return upcoming_fixtures[team]

In [16]:
#Returns the attacking difficulty of each upcoming fixture
def fixt_att_diff(team, look_ahead, weighting=1, current=gameweek):
    fixtures = get_upcoming_fixtures(team, look_ahead, current)

    opp_def_qual = fixtures.apply(lambda t: teams.at[t, "Defensive Quality"])
    self_att_qual = teams.at[team, "Attacking Quality"]
    difference = opp_def_qual.apply(lambda d: self_att_qual - d)

    att_diff = difference.apply(lambda x: x*weighting + 1)

    return att_diff

#Returns the defensive difficulty of each upcoming fixture
def fixt_def_diff(team, look_ahead, weighting=1, current=gameweek):
    fixtures = get_upcoming_fixtures(team, look_ahead, current)

    opp_att_qual = fixtures.apply(lambda t: teams.at[t, "Attacking Quality"])
    self_def_qual = teams.at[team, "Defensive Quality"]
    difference = opp_att_qual.apply(lambda d: self_def_qual - d)

    def_diff = difference.apply(lambda x: x*weighting + 1)

    return def_diff

In [17]:
def att_mult(team, look_ahead, weighting=1, current=gameweek):
    return fixt_att_diff(team, look_ahead, weighting, current).sum() 

def def_mult(team, look_ahead, weighting=1, current=gameweek):
    return fixt_def_diff(team, look_ahead, weighting, current).sum()

In [18]:
def app_mult(player_row, look_ahead, weighting=1, current=gameweek):
    team = player_row["Team"]
    pos = player_row["Pos."]

    if pos in ["GKP", "DEF"]:
        mult = def_mult(team, look_ahead, weighting, current)
    else:
        mult = att_mult(team, look_ahead, weighting, current)

    return player_row["Quality"] * mult

In [19]:
def top_players(weighting=1, look_ahead=5, current=gameweek):
    post_players = players_verbose
    post_players["Post Quality"] = players_verbose.apply(lambda row: app_mult(row, look_ahead, weighting, current), axis=1)
    post_players = post_players[['Post Quality', 'Quality', 'Pos.', 'Team', 'Price']]
    return post_players.sort_values("Post Quality", ascending=False)

In [20]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [21]:
def gen_team(weighting=1, look_ahead=5, current=gameweek):
    pls = top_players(weighting, look_ahead, current)
    pls = pls.groupby("Team").head(3).sort_values("Post Quality", ascending=False)
    pls["Fixture Mult."] = (pls["Post Quality"] / pls["Quality"])/5

    gkps = pls.loc[pls["Pos."] == "GKP"]
    defs = pls.loc[pls["Pos."] == "DEF"]
    mids = pls.loc[pls["Pos."] == "MID"]
    fwds = pls.loc[pls["Pos."] == "FWD"]

    return pd.concat([gkps.head(2), defs.head(5), mids.head(5), fwds.head(3)])

In [22]:
def gen_differentials(weighting=100):
    differentials = gen_team(weighting).merge(gen_team(), how="left", indicator=True, right_index=True, left_index=True)
    differentials = differentials[differentials["_merge"] == "left_only"][["Post Quality_x", "Quality_x", "Pos._x", "Team_x", "Price_x"]]
    differentials.columns = ["Post Quality", "Quality", "Pos.", "Team", "Price"]
    differentials["Post Quality"] = differentials["Post Quality"] / weighting
    return differentials

In [23]:
def bargains(max, weighting=1, look_ahead=5, current=gameweek):
    pls = top_players(weighting, look_ahead, current)
    pls = pls.loc[pls["Price"] <= max]
    return pls.head(15)

In [24]:
def get_squad(consider, cheapest_player):
    squad = pd.DataFrame(columns=["Post Quality", "Quality", "Pos.", "Team", "Price"], index=consider.index)

    allowed_per_position = {"GKP": 2, "DEF": 5, "MID": 5, "FWD": 3}

    size = 0
    for idx, row in consider.iterrows():

        squad_price = squad["Price"].sum()
        
        overbudget = False
        if size != 0:
            overbudget = (100 - (squad_price + row["Price"]))/(15-size) < cheapest_player

        try:
            team_count = squad["Team"].value_counts().loc[row["Team"]]
        except KeyError:
            team_count = 0

        try:
            pos_count = squad["Pos."].value_counts().loc[row["Pos."]]
        except KeyError:
            pos_count = 0

        if overbudget:
            next
        elif team_count == 3:
            next
        elif pos_count == allowed_per_position[row["Pos."]]:
            next

        else:
            size = size + 1
            squad.loc[idx] = row

        if size >= 15:
            break

    return squad.dropna()

In [25]:
def all_best_squad(threshold, cheapest_player, weighting=1, look_ahead=5, current=gameweek):
    cheap = bargains(cheapest_player, weighting, look_ahead, current)
    top = top_players(weighting, look_ahead, current).head(threshold)

    consider = pd.concat([cheap, top]).sort_values("Post Quality", ascending=False)
    consider.drop_duplicates(inplace=True)

    squad = get_squad(consider, cheapest_player)

    return squad


In [26]:
for i in range(200, 300, 10):
    squad = all_best_squad(i, 4)
    print(f"Weight: {i}, Quality: {squad['Post Quality'].sum()}, Price: {squad['Price'].sum()}")

Weight: 200, Quality: 489.1057500591199, Price: 92
Weight: 210, Quality: 489.1057500591199, Price: 92
Weight: 220, Quality: 489.1057500591199, Price: 92
Weight: 230, Quality: 489.1057500591199, Price: 92
Weight: 240, Quality: 489.1057500591199, Price: 92
Weight: 250, Quality: 501.0647797696132, Price: 96
Weight: 260, Quality: 501.0647797696132, Price: 96
Weight: 270, Quality: 501.0647797696132, Price: 96
Weight: 280, Quality: 501.0647797696132, Price: 96
Weight: 290, Quality: 501.0647797696132, Price: 96


In [27]:
all_best_squad(290, 4)

,Post Quality,Quality,Pos.,Team,Price
Player,,,,,
Salah,56.64,6.26,MID,LIV,12
Haaland,55.06,6.00,FWD,MCI,11
Alexander-Arnold,50.71,5.54,DEF,LIV,8
Chillwell,48.05,6.40,DEF,CHE,6
Cancelo,47.09,5.12,DEF,MCI,7
van Dijk,45.55,4.98,DEF,LIV,6
Laporte,39.95,4.35,DEF,MCI,6
Son,38.45,5.43,MID,TOT,10
Mount,29.16,3.69,MID,CHE,6


In [30]:
all_best_squad(290, 4, current=5)

,Post Quality,Quality,Pos.,Team,Price
Player,,,,,
Haaland,54.01,6.00,FWD,MCI,11
Salah,52.34,6.26,MID,LIV,12
Alexander-Arnold,49.51,5.54,DEF,LIV,8
Chillwell,46.87,6.40,DEF,CHE,6
Cancelo,46.23,5.12,DEF,MCI,7
van Dijk,44.47,4.98,DEF,LIV,6
Laporte,39.22,4.35,DEF,MCI,6
Son,36.42,5.43,MID,TOT,10
Mendy,26.70,3.65,GKP,CHE,6


In [28]:
def cheap_gkp():

    cheap = bargains(4)
    top = top_players().head(290)

    consider = pd.concat([cheap, top]).sort_values("Post Quality", ascending=False)
    consider.drop_duplicates(inplace=True)

    consider = consider.drop(consider[(consider["Pos."] == "GKP") & (consider["Price"] > 5)].index)

    #consider.loc[consider["Team"] == "CHE"]

    return get_squad(consider, 4)

,Post Quality,Quality,Pos.,Team,Price
Player,,,,,
Salah,56.64,6.26,MID,LIV,12
Haaland,55.06,6.00,FWD,MCI,11
Alexander-Arnold,50.71,5.54,DEF,LIV,8
Chillwell,48.05,6.40,DEF,CHE,6
Cancelo,47.09,5.12,DEF,MCI,7
van Dijk,45.55,4.98,DEF,LIV,6
Laporte,39.95,4.35,DEF,MCI,6
Son,38.45,5.43,MID,TOT,10
Mount,29.16,3.69,MID,CHE,6


In [29]:
consider = top_players().groupby("Team").head(3).drop_duplicates()
get_squad(consider, 5)

,Post Quality,Quality,Pos.,Team,Price
Player,,,,,
Salah,56.64,6.26,MID,LIV,12
Haaland,55.06,6.00,FWD,MCI,11
Alexander-Arnold,50.71,5.54,DEF,LIV,8
Chillwell,48.05,6.40,DEF,CHE,6
Cancelo,47.09,5.12,DEF,MCI,7
van Dijk,45.55,4.98,DEF,LIV,6
Dunk,22.83,3.60,DEF,BHA,5
Mbeumo,22.08,3.49,MID,BRE,5
Gallagher,17.38,3.39,MID,CRY,5


In [36]:
pls = top_players()
pls = pls.groupby("Team").head(3).sort_values("Post Quality", ascending=False)
pls["Fixture Mult."] = (pls["Post Quality"] / pls["Quality"])/5


pls = pls.groupby("Team").head(1)
pls.sort_values("Fixture Mult.", ascending=False)

,Post Quality,Quality,Pos.,Team,Price,Fixture Mult.
Player,,,,,,
Haaland,55.06,6.00,FWD,MCI,11,1.84
Salah,56.64,6.26,MID,LIV,12,1.81
Chillwell,48.05,6.40,DEF,CHE,6,1.50
Son,38.45,5.43,MID,TOT,10,1.42
Saka,27.75,4.00,MID,ARS,6,1.39
Coady,23.19,3.45,DEF,WOL,6,1.35
Vardy,27.68,4.24,FWD,LEI,10,1.31
Cucurella,23.09,3.64,DEF,BHA,6,1.27
Toney,22.25,3.51,FWD,BRE,6,1.27
